In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler



## Loading dataset

In [ ]:
csv_path = "./data"



csv_path = Path(csv_path)
df_total = pl.read_csv(csv_path)

In [ ]:
col_exp_vars = ["orientation", "density", "diameter"]
col_obj_vars = ["conductivity"]

df_exp = df_total.select(col_exp_vars)
df_obj = df_total.select(col_obj_vars)


df_total.head()

In [ ]:
plt.figure(figsize=(12, 9))
sns.heatmap(
    df_total.to_pandas().corr(), annot=True, cmap="cividis", fmt=".2f", linewidths=0.5
)

## Data preprocessing

Here, the following data preprocessing is performed.
#### Step 1. Exclusion of outliers
An outlier is data that differs in trend from other data.
When outliers are included in the training data, the model tries to read their trends as well, which reduces generalization performance.

Therefore, they should be removed.

#### Step 2. Convert DataFrame to array
This is because label names are not necessary during training and it is sufficient to know which column corresponds to which column.

#### Step 3. Separate into training data and test data
Train on training data and test generalization performance on test data.

#### Step 4. Perform standardization
Standardization is to convert the mean to 0 and the variance to 1 within an j.
Specifically, the following equation is used for the conversion.
$$x_{j_{std}}[i] = \frac{x_{j}[i] - \mu_{j}}{\sqrt{\sigma^2{_{j}}}}$$
where $j$ is the item name, $\mu_{j}=\Sigma_{i=1}^{N} x_{j}[i] / N$ and $\sigma^2{_{j}}=\Sigma_{i=1}^{N} \left\{ x_{j}[i] - \mu_{j} \right\}^2 / N$ are the mean and variance in the item, respectively, and i is the index in the item.
The advantage of standardization is that it allows for comparison by matching item-by-item scales.
<details>
<summary> Proofs for mean value and variance after standardization </summary>

$$
\begin{aligned}
\mu_{j_{std}} &= \frac{1}{N} \Sigma_{i=1}^{N} x_{j_{std}}[i]\\
&= \frac{1}{N} \Sigma_{i=1}^{N} \frac{x_{j}[i] - \mu_{j}}{\sqrt{\sigma^2{_{j}}}}\\
&= \frac{1}{\sqrt{\sigma^2{_{j}}}} \left\{ 
    \frac{\Sigma_{i=1}^{N} x_{j}[i]}{N} - 
    \frac{\Sigma_{i=1}^{N} \mu_{j}}{N}
\right\}\\
&= \frac{1}{\sqrt{\sigma^2{_{j}}}} \left(
    \mu_{j} - \mu_{j}
\right)\\
&= 0\\
\end{aligned}
$$

$$
\begin{aligned}
{\sigma^2}_{j_{std}} &= \frac{1}{N } \Sigma_{i=1}^{N} \left\{ x_{j_{std}}[i] - \mu_{j_{std}} \right\}^2\\
&= \frac{1}{N } \Sigma_{i=1}^{N} \left\{ x_{j_{std}}[i] - 0  \right\}^2\\
&= \frac{1}{N } \Sigma_{i=1}^{N} \left\{ \frac{x_{j}[i] - \mu_{j}}{\sqrt{\sigma^2{_{j}}}} \right\}^2\\
&= \frac{1}{\sigma^2{_{j}}} \frac{1}{N } \Sigma_{i=1}^{N} \left\{ x_{j}[i] - \mu_{j} \right\}^2\\
&= \frac{1}{\sigma^2{_{j}}} \cdot \sigma^2{_{j}}\\
&= 1\\
\end{aligned}
$$
</details>

In [ ]:
# Step 1: Exclusion of outliers
quantiles = df_exp.quantile(0.95)

df_prsd = df_total.filter(
    [pl.col(col) < quantiles.select(col) for col in col_exp_vars]
)
df_prsd

# Step 2: Convert DaraFrame to array
X = df_prsd.select(pl.exclude(col_obj_vars)).to_numpy()
y = df_prsd.select(col_obj_vars).to_numpy()

# Step 3: Separate into training data and test data
train_x, test_x, train_y, test_y = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Step 4: Perform standardization
scaler = StandardScaler()
scaler.fit(train_x)
train_x_scaled = scaler.transform(train_x)

## Model definition, training, and value prediction
The following equation is a model for multiple linear regression.
$$
\begin{aligned}
y_{pred} &= w_0 + w_1 \cdot x_1 + \cdots + w_D \cdot x_D\\
&= w_0 + \Sigma_{j=1}^{D} w_j \cdot x_j\\
&= \boldsymbol{w}^T \bm{x}
\end{aligned}\\
\boldsymbol{w} = \left[ w_0 \ w_1 \ w_2 \ \cdots w_D \right]^T, 
\boldsymbol{x} = \left[ 1 \ x_1 \ x_2 \cdots x_D \right]^T
$$
where $x_{j}$ is the value of the $j$ th item and $w_j$ is the weight parameter (partial regression coefficient) of each item that is updated by training.

Mean squared error (MSE) and gradient descent are used to update the parameters.
The MSE represents the error between the true value and the predicted value, and the parameters are updated using the gradient descent method to reduce the error.

MSE
$$
E = \frac{1}{N}\Sigma_{i=1}^{N} \left( 
    y_{pred_i} - y_{ture_i}
\right) ^ 2 \\
$$

Gradient Descent
$$
w \leftarrow w - \eta \nabla_{\boldsymbol{w}}E\\
$$
where, $\eta$ is learning rate.
<figure>
    <center>
        <img src="./figures/gradient_descent.svg" alt="The idea of the gradient descent" title="The idea of the gradient descenthe">
        <figcaption>
            The gradient descent method is based on the fact that the gradient of a continuous function represents the direction of the minima.
        </figcaption>
    </center>
</figure>

In [ ]:
# Definition
model = LinearRegression()
# training
model.fit(train_x_scaled, train_y)
# Value prediction
y_pred = model.predict(train_x_scaled)

## Checking Partial Regression Coefficients
The following values is the partial regression coefficients of each item.

In [ ]:
for xi, wi in zip(col_exp_vars, model.coef_[0]):
    print("{0:7}: {1:6.3f}".format(xi, wi))

Visualization using bar charts.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
ax.bar(
    col_exp_vars,
    model.coef_[0],
    edgecolor="black",
    facecolor="None",
    hatch="....."
)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.set_ylabel("Partial regression coefficient")
ax.set_xticks(range(len(col_exp_vars)), col_exp_vars, rotation=45)
ax.set_ylim(-1, 1)
ax.grid(which="both", axis="y")
ax.tick_params(axis="y", direction="in")
# ax.tick_params(axis="x", direction="in")

ax.set_axisbelow(True)

## Validate of the model
Refer to coefficient of determination ($R^2$) and MSE

If $R^2$ exceeds 0.5, it can be said that some trend has been obtained.

If the difference between the test data and the training data is large for MSE, it means that overfitting has occured and generalization performance is low.

In [ ]:
r2 = model.score(train_x_scaled, train_y)
mse_train = mean_squared_error(train_y, y_pred)
y_pred_test = model.predict(scaler.transform(test_x))
mse_test = mean_squared_error(test_y, y_pred_test)
print("Coefficient of determination: ", r2)
print(f"Mean squared error of training data: {mse_train: 0.4f}")
print(f"Mean squared error of test data: {mse_test: 0.4f}")